In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv("/content/drive/MyDrive/ml_datasets/incomeData.csv")
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
data.shape

(32561, 15)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  Income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## first of all let's handle missing data

In [8]:
data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
Income            0
dtype: int64

In [9]:
data['workclass'].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

## we might have many '?' values in data now we need to handle it

In [10]:
data.isin([' ?']).sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
Income               0
dtype: int64

## remove all rows in which '?' value is there

In [11]:
data = data.drop(data[data['workclass'] == ' ?'].index)
data = data.drop(data[data['occupation'] == ' ?'].index)
data = data.drop(data[data['native-country'] == ' ?'].index)
data.shape

(30162, 15)

## now reindexing

In [12]:
data.reset_index(drop=True, inplace=True)

In [13]:
data.tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income
30157,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
30158,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
30159,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
30160,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
30161,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


## handling nominal categorical data columns

In [14]:
data = pd.get_dummies(data, columns=['workclass', 'marital-status', 'occupation', 'relationship', 'sex'], drop_first=True)
data.head()

,age,fnlwgt,education,education-num,race,capital-gain,capital-loss,hours-per-week,native-country,Income,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,sex_ Male
0,39,77516,Bachelors,13,White,2174,0,40,United-States,<=50K,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,50,83311,Bachelors,13,White,0,0,13,United-States,<=50K,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,38,215646,HS-grad,9,White,0,0,40,United-States,<=50K,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,53,234721,11th,7,Black,0,0,40,United-States,<=50K,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,28,338409,Bachelors,13,Black,0,0,40,Cuba,<=50K,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


## education columns we can consider it is ordinal categoricald data so we can handle it that way

In [15]:
data['education'] = data['education'].map({' Preschool':1, ' 1st-4th':2, ' 5th-6th':3, ' 7th-8th':4, 
                               ' 9th':5, ' 10th':6, ' 11th':7, ' 12th':8, ' HS-grad':9,
                               ' Prof-school':10, ' Assoc-acdm':11, ' Assoc-voc':12, ' Some-college':13,
                               ' Bachelors':14, ' Masters':15, ' Doctorate':16,
                              })
data.shape

(30162, 41)

## now replace >50k and <50k data with 0 and 1

In [16]:
data['Income'] = data['Income'].map({' <=50K': 0, ' >50K':1})

In [17]:
data.head(2)

,age,fnlwgt,education,education-num,race,capital-gain,capital-loss,hours-per-week,native-country,Income,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,sex_ Male
0,39,77516,14,13,White,2174,0,40,United-States,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,50,83311,14,13,White,0,0,13,United-States,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## removing not used columns

In [18]:
data = data.drop(columns=['fnlwgt', 'race', 'capital-gain', 'capital-loss', 'native-country'])

In [19]:
## spitting X and Y values
X = data.drop('Income', axis=1)
y = data['Income']

## data seems imbalanced we need to make them balanced

In [20]:
from imblearn.combine import SMOTETomek
#smotetomek does both downsampling and upsampling of class

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [21]:
smote = SMOTETomek(0.9)
X, y = smote.fit_sample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [22]:
from collections import Counter
Counter(y)

Counter({0: 22200, 1: 19934})

## Doing train_test_split

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Implementing RandomForest

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
forest = RandomForestClassifier()

In [ ]:
forest_grid_params = {'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                     'min_samples_split': [x for x in range(2,5)],
                     'max_leaf_nodes': [2, 3, 4, None],
                     'min_samples_leaf': [x for x in range(1,5)],
                     'n_estimators' : [90, 100, 110, 120, 130,140, 150, 160, 170, 180, 190, 200],
                     }

In [ ]:
forest_cv = RandomizedSearchCV(forest, forest_grid_params,n_iter=400, n_jobs=-1, cv=5, verbose=2)

In [ ]:
forest_cv.fit(X_train, y_train)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 18.0min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 29.6min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 47.9min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 70.6min
[Parallel(n_jobs=-1)]: Done 1981 tasks      | elapsed: 92.2min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed: 93.2min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=400,
                   n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_leaf_nodes': [2, 3, 4, None],
                                        'min_samples_leaf': [1, 2, 3, 4],
                                        'min_samples_split': [2, 3, 4],
                                        'n_estimators': [90, 100, 110, 120, 130,
                                                         140, 150, 160, 170,
                                                         180, 190, 200]},
                   verbose=2)

In [ ]:
forest_cv.best_score_

0.8723177124858121

In [ ]:
forest_cv.best_estimator_

RandomForestClassifier(min_samples_leaf=2, n_estimators=190)

In [ ]:
forest = RandomForestClassifier(min_samples_leaf=2, n_estimators=190)

In [ ]:
forest.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=2, n_estimators=190)

In [ ]:
forest_y_predicted = forest.predict(X_test)

In [ ]:
sc = accuracy_score(y_test, forest_y_predicted)
sc

0.8721990125332321

In [ ]:
confusion_matrix(y_test, forest_y_predicted)

array([[4889,  596],
       [ 503, 4530]], dtype=int64)

In [ ]:
print(classification_report(y_test, forest_y_predicted))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      5485
           1       0.88      0.90      0.89      5033

    accuracy                           0.90     10518
   macro avg       0.90      0.90      0.90     10518
weighted avg       0.90      0.90      0.90     10518



In [ ]:
Pkl_Filename = "Income_Model_Random_Forest.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(forest, file)

In [ ]:
import pickle
model = pickle.load(open("Income_Model_Random_Forest.pkl", "rb"))

In [ ]:
predicted = model.predict([[40,9,9,40,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]])
predicted

array([0], dtype=int64)

In [32]:
from xgboost import XGBClassifier

In [26]:
xgboost_clf = XGBClassifier().fit(X_train, y_train)

In [27]:
xgb_y_predicted = xgboost_clf.predict(X_test)

In [28]:
accuracy_score(y_test, xgb_y_predicted)

0.8520030377824188

In [29]:
confusion_matrix(y_test, xgb_y_predicted)

array([[4522, 1028],
       [ 531, 4453]])

In [30]:
param_grids_xbg = {"learning_rate" : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
              "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
              "min_child_weight" : [ 1, 3, 5, 7 ],
              "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
              "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

In [34]:
xgb_tuning = RandomizedSearchCV(xgboost_clf, param_grids_xbg, n_iter=400, n_jobs=-1, cv=5, verbose=2)

In [35]:
xgb_tuning.fit(X_train, y_train)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 40.1min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 58.1min
[Parallel(n_jobs=-1)]: Done 1981 tasks      | elapsed: 79.3min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed: 80.1min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, sc...
                                           seed=None, silent=None, subsample=1,
                                           verbosity=1),
                   iid='deprecated', n_i

In [38]:
xgb_tuning.best_score_

0.8720569620253166

In [39]:
xgb_tuning.best_params_

{'colsample_bytree': 0.5,
 'gamma': 0.3,
 'learning_rate': 0.1,
 'max_depth': 12,
 'min_child_weight': 1}

In [40]:
xgboost_clf = XGBClassifier(colsample_bytree=0.5, gamma=0.3, learning_rate=0.1, max_depth=12, min_child_weight=1)

In [41]:
xgboost_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.3,
              learning_rate=0.1, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [42]:
xgb_predicted = xgboost_clf.predict(X_test)

In [43]:
confusion_matrix(y_test, xgb_predicted)

array([[4725,  825],
       [ 494, 4490]])

In [44]:
accuracy_score(y_test, xgb_predicted)

0.8747864059236757

In [47]:
import pickle
file_name = '/content/drive/MyDrive/ml/income-model/xgboost.pkl'
with open(file_name, 'wb') as file:  
    pickle.dump(xgboost_clf, file)